# 3. Solutions to GTFS Reconstruction

This notebook demonstrates converting optimization solutions back to GTFS feeds. It covers:

3.1 Quick Problem Setup
- Load GTFS data and create optimization problem
- Run a simple optimization to get solutions

3.2 Phase 1: Solution Conversion
- Convert solution matrices to headway values
- Validate solutions and extract templates

3.3 Phase 2: GTFS Reconstruction
- Generate new trips and stop times
- Create complete GTFS feed

3.4 Validation and Analysis
- Compare original vs reconstructed GTFS
- Verify service patterns match optimization

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Add src to path
project_root = Path.cwd().parent
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))


print("=== GTFS RECONSTRUCTION WORKFLOW ===")
print("🔄 Testing solution-to-GTFS conversion pipeline")
print(f"📁 Project root: {project_root}")

=== GTFS RECONSTRUCTION WORKFLOW ===
🔄 Testing solution-to-GTFS conversion pipeline
📁 Project root: /home/hussein/Documents/GitHub/transit_opt


## 3.1 Quick Problem Setup

Create a minimal optimization problem to generate test solutions.

In [2]:
from transit_opt.preprocessing.prepare_gtfs import GTFSDataPreparator
from transit_opt.optimisation.config.config_manager import OptimizationConfigManager
from transit_opt.optimisation.runners.pso_runner import PSORunner

print("=== LOADING GTFS DATA ===")

# Quick GTFS setup
preparator = GTFSDataPreparator(
    gtfs_path='../data/external/study_area_gtfs_bus.zip',
    interval_hours=6,  # 4 periods per day for simplicity
    date=None,
    turnaround_buffer=1.15,
    max_round_trip_minutes=240.0,
    no_service_threshold_minutes=480.0,
    log_level="INFO"
)

# Simple headway choices for testing
allowed_headways = [10, 15, 30, 60, 120]
opt_data = preparator.extract_optimization_data(allowed_headways)

print(f"✅ Loaded {opt_data['n_routes']} routes, {opt_data['n_intervals']} intervals")
print(f"📊 Headway choices: {allowed_headways}")
print(f"🎯 Decision variables: {np.prod(opt_data['decision_matrix_shape'])}")

2025-09-23 19:00:58,143 - transit_opt.preprocessing.prepare_gtfs - INFO - Initializing GTFSDataPreparator with 6h intervals
2025-09-23 19:00:58,144 - transit_opt.preprocessing.prepare_gtfs - INFO - Loading GTFS feed from ../data/external/study_area_gtfs_bus.zip


=== LOADING GTFS DATA ===


2025-09-23 19:01:00,504 - transit_opt.preprocessing.prepare_gtfs - INFO - Using full GTFS feed (all service periods)
2025-09-23 19:01:01,881 - transit_opt.preprocessing.prepare_gtfs - INFO - GTFS loaded and cached in 3.74 seconds
2025-09-23 19:01:01,882 - transit_opt.preprocessing.prepare_gtfs - INFO - Dataset: 13,974 trips, 703,721 stop times
2025-09-23 19:01:01,882 - transit_opt.preprocessing.prepare_gtfs - INFO - Extracting optimization data with 5 allowed headways
2025-09-23 19:01:01,883 - transit_opt.preprocessing.prepare_gtfs - INFO - Extracting route essentials with 6-hour intervals
2025-09-23 19:01:02,846 - transit_opt.preprocessing.prepare_gtfs - WARNING - Route 73302: Round-trip 317.4min exceeds limit (240.0min), filtered out
2025-09-23 19:01:02,881 - transit_opt.preprocessing.prepare_gtfs - WARNING - Route 54721: Round-trip 366.8min exceeds limit (240.0min), filtered out
2025-09-23 19:01:04,128 - transit_opt.preprocessing.prepare_gtfs - WARNING - Route 30922: Round-trip 247.

✅ Loaded 147 routes, 4 intervals
📊 Headway choices: [10, 15, 30, 60, 120]
🎯 Decision variables: 588


In [3]:
# Quick optimization to get test solutions
print("=== RUNNING QUICK OPTIMIZATION ===")

# Minimal config for fast testing
test_config = {
    'problem': {
        'objective': {
            'type': 'HexagonalCoverageObjective',
            'spatial_resolution_km': 5.0,  # Larger zones for speed
            'crs': 'EPSG:3857'
        },
        'constraints': [
            {
                'type': 'FleetTotalConstraintHandler',
                'baseline': 'current_peak',
                'tolerance': 0.3,  # Relaxed for testing
                'measure': 'peak'
            }
        ]
    },
    'optimization': {
        'algorithm': {
            'type': 'PSO',
            'pop_size': 20,  # Small for speed
            'inertia_weight': 0.9,
            'inertia_weight_final': 0.4,
            'cognitive_coeff': 2.0,
            'social_coeff': 2.0,
            'use_penalty_method': False
        },
        'termination': {
            'max_generations': 10  # Very short for testing
        },
        'monitoring': {
            'progress_frequency': 5,
            'save_history': True,
            'detailed_logging': False
        }
    }
}

# Run optimization
config_manager = OptimizationConfigManager(config_dict=test_config)
pso_runner = PSORunner(config_manager)
result = pso_runner.optimize(opt_data, track_best_n=5)

print(f"✅ Optimization complete: {result.best_objective:.4f}")
print(f"📊 Found {len(result.best_feasible_solutions) if hasattr(result, 'best_feasible_solutions') and result.best_feasible_solutions else 0} feasible solutions")
print(f"⏱️  Time: {result.optimization_time:.1f}s")

=== RUNNING QUICK OPTIMIZATION ===
📋 Using provided configuration dictionary
🚀 STARTING PSO OPTIMIZATION
🗺️ Setting up spatial analysis with 5.0km resolution
🗺️  Reprojected 6897 stops to EPSG:3857
🔧 Creating 88 × 156 = 13728 grid cells
   Grid bounds: (-454051, 6583019) to (-14085, 7359492) meters
   Cell size: 5000.0m × 5000.0m
✅ Created 13728 hexagonal zones in EPSG:3857
🚀 Using spatial join for zone mapping...
✅ Mapped 6897 stops to zones
🚀 Pre-computing route-stop mappings...
✅ Cached stops for 278 routes/services
✅ Spatial system ready: 13728 hexagonal zones
   📋 Creating 1 constraint handler(s)...
      Creating constraint 1: FleetTotalConstraintHandler
         ✓ FleetTotal: 1 constraint(s)
🏗️  CREATING TRANSIT OPTIMIZATION PROBLEM:
   📊 Problem dimensions:
      Routes: 147
      Time intervals: 4
      Headway choices: 6
   🔧 Pymoo parameters:
      Decision variables: 588
      Objectives: 1
      Constraints: 1
   🚦 Hard constraints: 1 constraint(s)
   📋 Constraint breakdow

## 3.2 Phase 1: Solution Conversion

Test the SolutionConverter class for converting optimization results to headway specifications.

In [4]:
from transit_opt.gtfs.gtfs import SolutionConverter

print("=== TESTING SOLUTION CONVERTER ===")

# Create converter
converter = SolutionConverter(opt_data)

# Test with best solution
test_solution = result.best_solution
print(f"🔍 Testing solution shape: {test_solution.shape}")

# Validate solution
validation = converter.validate_solution(test_solution)
print(f"\n✅ VALIDATION RESULTS:")
print(f"   Valid: {validation['valid']}")
print(f"   Errors: {len(validation['errors'])}")
print(f"   Warnings: {len(validation['warnings'])}")

if validation['errors']:
    for error in validation['errors']:
        print(f"   ❌ {error}")

# Show statistics
stats = validation['statistics']
print(f"\n📊 SOLUTION STATISTICS:")
print(f"   Service coverage: {stats['service_percentage']:.1f}%")
print(f"   Active cells: {stats['service_cells']}/{stats['total_cells']}")
print(f"   Headway distribution: {stats['headway_distribution']}")

=== TESTING SOLUTION CONVERTER ===
🔍 Testing solution shape: (147, 4)

✅ VALIDATION RESULTS:
   Valid: True
   Errors: 0
   Warnings: 0

📊 SOLUTION STATISTICS:
   Service coverage: 91.5%
   Active cells: 538/588
   Headway distribution: {'10min': 58, '15min': 119, '30min': 136, '60min': 114, '120min': 111, 'No Service': 50}


In [5]:
# Convert to headways dictionary
print("=== CONVERTING TO HEADWAYS ===")

headways_dict = converter.solution_to_headways(test_solution)
print(f"📋 Converted {len(headways_dict)} routes")

# Show example route
example_route = list(headways_dict.keys())[50]
print(f"\n📋 EXAMPLE ROUTE ({example_route}):")
for interval, headway in headways_dict[example_route].items():
    if headway is None:
        print(f"   {interval:>15}: No Service")
    else:
        print(f"   {interval:>15}: {headway:.0f} minutes")

# Extract template trips
print(f"\n🔧 EXTRACTING TEMPLATE TRIPS:")
templates = converter.get_template_trips()
print(f"   Templates extracted: {len(templates)}/{len(headways_dict)}")

if templates:
    example_template = templates[example_route]
    print(f"   Example template ({example_route}):")
    print(f"      Trip ID: {example_template['trip_id']}")
    print(f"      Duration: {example_template['duration_minutes']:.1f} min")
    print(f"      Stops: {example_template['n_stops']}")

=== CONVERTING TO HEADWAYS ===
📋 Converted 147 routes

📋 EXAMPLE ROUTE (30532):
            00-06h: 60 minutes
            06-12h: 15 minutes
            12-18h: 10 minutes
            18-24h: 120 minutes

🔧 EXTRACTING TEMPLATE TRIPS:
   Templates extracted: 147/147
   Example template (30532):
      Trip ID: VJ7a7e72c601331a8a0a26ade124264eb58cdd5889
      Duration: 63.0 min
      Stops: 62


## 3.3 Phase 2: GTFS Reconstruction 

This section will implement trip generation and GTFS file creation.

In [7]:
# Add this new cell for Phase 2 testing
print("=== PHASE 2: TRIP GENERATION ===")

# Generate trips and stop times
new_trips_df, new_stop_times_df = converter.generate_trips_and_stop_times(headways_dict, templates)

print(f"📊 GENERATION RESULTS:")
print(f"   Generated trips: {len(new_trips_df)}")
print(f"   Generated stop times: {len(new_stop_times_df)}")

if len(new_trips_df) > 0:
    print(f"   Routes with trips: {new_trips_df['route_id'].nunique()}")
    print(f"   Trips per route (avg): {len(new_trips_df) / new_trips_df['route_id'].nunique():.1f}")
    
    # Show example trip
    example_trip_id = new_trips_df.iloc[5]['trip_id']
    example_route_id = new_trips_df.iloc[5]['route_id']
    
    print(f"\n📋 EXAMPLE TRIP ({example_trip_id}):")
    print(f"   Route: {example_route_id}")
    
    # Show stop times for this trip
    trip_stop_times = new_stop_times_df[new_stop_times_df['trip_id'] == example_trip_id]
    print(f"   Stops: {len(trip_stop_times)}")
    print(f"   First departure: {trip_stop_times.iloc[0]['departure_time']}")
    print(f"   Last arrival: {trip_stop_times.iloc[-1]['arrival_time']}")
    
    # Validate timing
    first_dep = trip_stop_times.iloc[0]['departure_seconds']
    last_arr = trip_stop_times.iloc[-1]['arrival_seconds']
    actual_duration = (last_arr - first_dep) / 60
    template_duration = templates[example_route_id]['duration_minutes']
    
    print(f"   Duration: {actual_duration:.1f} min (template: {template_duration:.1f} min)")

=== PHASE 2: TRIP GENERATION ===
📊 GENERATION RESULTS:
   Generated trips: 6962
   Generated stop times: 297387
   Routes with trips: 147
   Trips per route (avg): 47.4

📋 EXAMPLE TRIP (opt_trip_50627_5):
   Route: 50627
   Stops: 43
   First departure: 00:50:00
   Last arrival: 01:25:00
   Duration: 35.0 min (template: 35.0 min)


In [8]:
# Add this diagnostic cell to see the current structure
print("=== DATA STRUCTURE ANALYSIS ===")

print(f"📊 OPTIMIZATION DATA STRUCTURE:")
print(f"   Routes: {len(opt_data['routes']['ids'])} unique route IDs")
print(f"   Intervals: {len(opt_data['intervals']['labels'])} time periods")
print(f"   Headway choices: {opt_data['allowed_headways']}")

print(f"\n📋 HEADWAYS DICTIONARY STRUCTURE:")
print(f"   Format: route_id -> {{interval_label: headway_minutes}}")
example_route = list(headways_dict.keys())[0]
print(f"   Example ({example_route}): {headways_dict[example_route]}")

print(f"\n🚌 TEMPLATES STRUCTURE:")
print(f"   Format: route_id -> {{trip_id, duration_minutes, n_stops, stop_times_df}}")
if templates:
    example_template = templates[example_route]
    print(f"   Example ({example_route}):")
    print(f"      - trip_id: {example_template['trip_id']}")
    print(f"      - duration: {example_template['duration_minutes']:.1f} min")
    print(f"      - stops: {example_template['n_stops']}")

print(f"\n🎯 GENERATED DATA STRUCTURE:")
print(f"   new_trips_df columns: {list(new_trips_df.columns)}")
print(f"   new_stop_times_df columns: {list(new_stop_times_df.columns)}")

# Show interval breakdown
print(f"\n⏰ INTERVAL STRUCTURE:")
for i, (label, (start, end)) in enumerate(zip(opt_data['intervals']['labels'], opt_data['intervals']['hours'])):
    print(f"   {i}: {label} = {start:02d}:00 to {end:02d}:00")

=== DATA STRUCTURE ANALYSIS ===
📊 OPTIMIZATION DATA STRUCTURE:
   Routes: 147 unique route IDs
   Intervals: 4 time periods
   Headway choices: [  10.   15.   30.   60.  120. 9999.]

📋 HEADWAYS DICTIONARY STRUCTURE:
   Format: route_id -> {interval_label: headway_minutes}
   Example (50627): {'00-06h': 10.0, '06-12h': 10.0, '12-18h': 60.0, '18-24h': 120.0}

🚌 TEMPLATES STRUCTURE:
   Format: route_id -> {trip_id, duration_minutes, n_stops, stop_times_df}
   Example (50627):
      - trip_id: VJ61453c073b835840fdf78144f238befa6c687ca8
      - duration: 35.0 min
      - stops: 43

🎯 GENERATED DATA STRUCTURE:
   new_trips_df columns: ['trip_id', 'route_id', 'service_id', 'trip_headsign', 'direction_id']
   new_stop_times_df columns: ['trip_id', 'stop_id', 'stop_sequence', 'arrival_time', 'departure_time', 'arrival_seconds', 'departure_seconds', 'pickup_type', 'drop_off_type']

⏰ INTERVAL STRUCTURE:
   0: 00-06h = 00:00 to 06:00
   1: 06-12h = 06:00 to 12:00
   2: 12-18h = 12:00 to 18:00
   

In [9]:
# Add this cell to understand trip generation
print("=== TRIP GENERATION ANALYSIS ===")

# Analyze trips per route and interval
route_analysis = []
for route_id in new_trips_df['route_id'].unique()[:5]:  # Check first 5 routes
    route_trips = new_trips_df[new_trips_df['route_id'] == route_id]
    route_headways = headways_dict[route_id]
    
    print(f"\n🚌 ROUTE {route_id}:")
    print(f"   Total trips generated: {len(route_trips)}")
    print(f"   Headway schedule: {route_headways}")
    
    # Calculate expected trips per interval
    for interval_label, headway in route_headways.items():
        if headway is not None:
            interval_duration_hours = 6  # Your interval_hours setting
            expected_trips = (interval_duration_hours * 60) / headway
            print(f"   {interval_label}: {headway}min headway → ~{expected_trips:.1f} trips expected")

# Show trip ID patterns
print(f"\n🎫 TRIP ID PATTERNS:")
sample_trips = new_trips_df[new_trips_df['route_id'] == list(new_trips_df['route_id'].unique())[0]]
for trip_id in sample_trips['trip_id'].head(8):
    print(f"   {trip_id}")

=== TRIP GENERATION ANALYSIS ===

🚌 ROUTE 50627:
   Total trips generated: 75
   Headway schedule: {'00-06h': 10.0, '06-12h': 10.0, '12-18h': 60.0, '18-24h': 120.0}
   00-06h: 10.0min headway → ~36.0 trips expected
   06-12h: 10.0min headway → ~36.0 trips expected
   12-18h: 60.0min headway → ~6.0 trips expected
   18-24h: 120.0min headway → ~3.0 trips expected

🚌 ROUTE 50628:
   Total trips generated: 16
   Headway schedule: {'00-06h': 30.0, '06-12h': 120.0, '12-18h': None, '18-24h': 120.0}
   00-06h: 30.0min headway → ~12.0 trips expected
   06-12h: 120.0min headway → ~3.0 trips expected
   18-24h: 120.0min headway → ~3.0 trips expected

🚌 ROUTE 50629:
   Total trips generated: 62
   Headway schedule: {'00-06h': 10.0, '06-12h': 30.0, '12-18h': 60.0, '18-24h': 15.0}
   00-06h: 10.0min headway → ~36.0 trips expected
   06-12h: 30.0min headway → ~12.0 trips expected
   12-18h: 60.0min headway → ~6.0 trips expected
   18-24h: 15.0min headway → ~24.0 trips expected

🚌 ROUTE 58940:
   Tota

In [ ]:
print("=== TRIP GENERATION VERIFICATION (FIXED) ===")

test_route = list(headways_dict.keys())[7]
test_headways = headways_dict[test_route]
test_trips = new_trips_df[new_trips_df['route_id'] == test_route]

print(f"🔍 TRACING ROUTE {test_route}:")
print(f"   Headway schedule: {test_headways}")
print(f"   Generated trips: {len(test_trips)}")

# Group trips by time interval based on trip_id pattern
for interval_idx, (interval_label, headway) in enumerate(test_headways.items()):
    if headway is not None:
        start_hour = interval_idx * 6  # Your 6-hour intervals
        
        # Look for the correct pattern after fixing trip IDs
        interval_trips = test_trips[test_trips['trip_id'].str.contains(f"_{start_hour:02d}_")]
        
        expected_trips = (6 * 60) / headway  # 6 hours / headway
        print(f"   {interval_label} ({start_hour:02d}h): {len(interval_trips)} trips (expected ~{expected_trips:.1f})")
        
        # Show actual trip IDs for this interval
        if len(interval_trips) > 0:
            sample_ids = interval_trips['trip_id'].head(3).tolist()
            print(f"      Sample IDs: {sample_ids}")

=== TRIP GENERATION VERIFICATION ===
🔍 TRACING ROUTE 12826:
   Headway schedule: {'00-06h': 15.0, '06-12h': 30.0, '12-18h': 10.0, '18-24h': 30.0}
   Generated trips: 70
   00-06h (00h): 0 trips (expected ~24.0)
   06-12h (06h): 0 trips (expected ~12.0)
   12-18h (12h): 0 trips (expected ~36.0)
   18-24h (18h): 0 trips (expected ~12.0)
